# Задание

Будем работать с моделью BERT.
Цель задания — познакомиться с моделью на примере задачи Questions
Answering (ответы на вопросы). Для этого необходимо:
1. До установить нужные пакеты
2. Cоздать файл q&a.py в коде которого.
3. Загрузить пред обученную модель Bert
4. Задать контекст — это текст в котором модель будет искать ответы на те вопросы, которые  ей задают
5. Реализовать бота, который будет принимать строку ( вопрос) и
передавать его модели, а затем выведет на экран ответ.
Для создания такого бота с использованием BERT, мы будем использовать
библиотеку transformers от Hugging Face и framework flask для создания
веб-приложения. Этот бот будет обрабатывать
пользовательские запросы и отвечать на них с использованием модели BERT.
Конкретная BERT модель выбирается с учетом типа задачи, в данном случае
нужно использовать BertForQuestionAnswering, внутри у нее необходимая
архитектура, при этом голова
соответствует задаче ответов на вопросы.
После запуска кода выведет адрес web страницы, перейдя по нему,
должны увидеть web интерфейс.
В качестве контекста (параграфа) используем кусочек из Алисы в стране чудес,
он на английском, так как модель училась на этом языке.
Для проверки можно использовать следующие вопросы:
1. "Who did Alice follow down the rabbit hole?"
2. "What did Alice see as she was falling down the well?"
3. "How did Alice manage to shrink in size?"


In [11]:
# Установка нужных библиотек:
# transformers — для загрузки предобученной модели BERT,
# gradio — для создания простого веб-интерфейса.
!pip install -q transformers gradio

In [ ]:


# Импорт библиотек:
import torch  # Библиотека для работы с нейросетями и тензорами (массивами)
from transformers import BertTokenizer, BertForQuestionAnswering  # Загрузка токенизатора и модели BERT
import gradio as gr  # Простая библиотека для создания интерфейсов
import urllib.request  # Чтобы скачать текст книги с сайта
# Импортирует функцию truncate из модуля os.
# Эта функция используется для обрезки (усечения) файла до заданного размера (в байтах).
from os import truncate

In [13]:

# Определяем, какое устройство использовать:
# - Если доступна видеокарта (GPU) — используем её
# - Если доступен чип Apple (MPS) — используем его
# - Иначе используем обычный процессор (CPU)
device = (
    torch.device("cuda") if torch.cuda.is_available() else
    torch.device("cpu")
)


In [ ]:


# Загружаем предобученный токенизатор BERT.
# Он преобразует текст в числа (токены), понятные модели.
# Модель обучена отвечать на вопросы (SQuAD датасет).
tokenizer = BertTokenizer.from_pretrained(
    'bert-large-uncased-whole-word-masking-finetuned-squad'
)

# Загружаем саму модель BERT для задачи "вопрос-ответ".
# Она предсказует, где в тексте начинается и заканчивается ответ.
model = BertForQuestionAnswering.from_pretrained(
    'bert-large-uncased-whole-word-masking-finetuned-squad'
).to(device)  # Переносим модель на выбранное устройство (CPU, GPU)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:

# Скачиваем текст книги "Алиса в стране чудес" с сайта Project Gutenberg
url = "https://www.gutenberg.org/files/11/11-0.txt"
urllib.request.urlretrieve(url, "alice.txt")

# Читаем файл и соединяем все строки в один длинный текст
with open('alice.txt', 'r', encoding='utf-8') as file:
    full_text = ' '.join(file.readlines())

In [ ]:


# Функция для разбиения текста на части, чтобы не превышать лимит BERT
# BERT работает максимум с 512 токенами, поэтому мы используем запас — 400
def split_context(text, max_tokens=400):
    # Преобразуем весь текст в список токенов (чисел)
    tokens = tokenizer.encode(text, truncation=False)
    chunks = []  # список для хранения фрагментов текста
    # Разбиваем токены на части по max_tokens
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i + max_tokens]  # берём фрагмент
        chunks.append(tokenizer.decode(chunk))  # превращаем его обратно в текст в виде списка кусков исходного текста
    return chunks  # возвращаем список фрагментов

# Разбиваем текст книги один раз заранее
context_chunks = split_context(full_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (37027 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# Главная функция для ответа на вопросы
def get_answer(question):
    best_answer = ""  # сюда запишется лучший ответ
    # Устанавливаем начальное значение для лучшего найденного "скор" (оценки уверенности модели).
    # float('-inf') означает "минус бесконечность" — гарантирует, что первый настоящий результат будет лучше.
    best_score = float('-inf')  # начальное значение — минимально возможное

    # Проходим по каждому фрагменту текста
    for context in context_chunks:
        # Создаём вход для модели: пара "вопрос + фрагмент текста"
        inputs = tokenizer.encode_plus(
            question,       # вопрос
            context,        # кусок текста, в котором ищем ответ
            add_special_tokens=True,  # добавить служебные токены [CLS], [SEP]
            return_tensors='pt',      # вернуть PyTorch-тензоры
            truncation=True,          # обрезать, если слишком длинно
            max_length=512            # максимальная длина (ограничение BERT)
        )

        # Перемещаем входные данные на нужное устройство (CPU или GPU)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Пропускаем через модель без обучения (no_grad = без вычисления градиентов)
        with torch.no_grad():
            outputs = model(**inputs)

        # Модель возвращает два массива:
        # start_logits — вероятность начала ответа на каждом токене
        # end_logits — вероятность конца ответа на каждом токене
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        # Считаем "уверенность" модели как сумму наибольших значений
        score = start_logits.max() + end_logits.max()

        # Если модель уверена больше, чем раньше — сохраняем этот ответ
        if score > best_score:
            best_score = score
            answer_start = torch.argmax(start_logits)  # индекс начала
            answer_end = torch.argmax(end_logits) + 1  # индекс конца (+1 потому что end — включительно)
            input_ids = inputs['input_ids'][0]  # сами токены
            answer_tokens = input_ids[answer_start:answer_end]  # выделяем токены ответа
            best_answer = tokenizer.decode(answer_tokens)  # переводим токены обратно в текст

    # Если ответ пустой, вернём "Ответ не найден"
    return best_answer if best_answer.strip() else "Ответ не найден."

In [19]:
# Создаём простой веб-интерфейс с помощью Gradio:
# - fn=get_answer — функция, которая будет вызываться
# - inputs="text" — пользователь вводит текст (вопрос)
# - outputs="text" — выводим текст (ответ)
iface = gr.Interface(
    fn=get_answer,
    inputs="text",
    outputs="text",
    title="BERT Q&A on Alice in Wonderland",  # заголовок интерфейса
    description="Введите вопрос на английском языке — я найду ответ в книге 'Alice in Wonderland'"
)

# Запуск интерфейса. share=True позволяет получить ссылку на ваш интерфейс
iface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b24cce2290dd527f63.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Using existing dataset file at: .gradio/flagged/dataset1.csv
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://095894e630ff71cf2f.gradio.live
Killing tunnel 127.0.0.1:7860 <> https://40eb81694743812f3b.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://b24cce2290dd527f63.gradio.live


# вывод:

В коде реализована полноценная модель "вопрос–ответ" на базе BERT, с возможностью задавать вопросы к книге Alice in Wonderland через простой веб-интерфейс на Gradio.

В коде реализовано:

Загрузка предобученной модели bert-large-uncased (fine-tuned на SQuAD),

Разбивка текста книги на части (400 токенов) (для обхода ограничения в 512 токенов),

Поиск наиболее вероятного отрывока, содержащего ответ,

Отображение результата пользователю через веб-интерфейс gradio.